# 02. アノテーション変換 - Ground Truth出力 → YOLOv8-pose形式

## 概要
1. Ground Truthの出力マニフェスト（JSONL）を読み込み
2. キーポイントデータをYOLOv8-pose形式に変換
3. train / val / test に分割
4. 学習用ディレクトリ構造をS3に配置

## 1. セットアップ

In [ ]:
import boto3
import json
import os
import random
import shutil
from pathlib import Path
from PIL import Image
from io import BytesIO

import sagemaker

s3 = boto3.client('s3')
sess = sagemaker.Session()

SAGEMAKER_BUCKET = sess.default_bucket()
SOURCE_BUCKET = 'facteye-images-20251114'
REGION = 'us-east-1'

# ローカル作業ディレクトリ
WORK_DIR = Path('/tmp/yolo_dataset')
WORK_DIR.mkdir(parents=True, exist_ok=True)

# データ分割比率
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

# キーポイント定義（Ground Truthラベルとの対応）
KEYPOINT_LABELS = ['needle_tip', 'needle_center', 'scale_min', 'scale_max']
NUM_KEYPOINTS = len(KEYPOINT_LABELS)

print(f'作業ディレクトリ: {WORK_DIR}')
print(f'キーポイント数: {NUM_KEYPOINTS}')
print(f'分割比率: train={TRAIN_RATIO}, val={VAL_RATIO}, test={TEST_RATIO}')

## 2. Ground Truth 出力マニフェスト読み込み

In [ ]:
# Ground Truth ジョブ名を指定
GT_JOB_NAME = 'facteye-meter-keypoint-XXXXXXXX-XXXXXX'  # TODO: 実際のジョブ名に置換

# 出力マニフェストのS3パス
OUTPUT_MANIFEST_KEY = f'ground-truth/output/{GT_JOB_NAME}/manifests/output/output.manifest'

def load_ground_truth_manifest(bucket, key):
    """Ground Truthの出力マニフェスト（JSONL）を読み込む"""
    response = s3.get_object(Bucket=bucket, Key=key)
    content = response['Body'].read().decode('utf-8')
    
    entries = []
    for line in content.strip().split('\n'):
        if line.strip():
            entries.append(json.loads(line))
    
    return entries

manifest_entries = load_ground_truth_manifest(SAGEMAKER_BUCKET, OUTPUT_MANIFEST_KEY)
print(f'マニフェストエントリ数: {len(manifest_entries)}')

# サンプル表示
if manifest_entries:
    print('\nサンプルエントリ:')
    print(json.dumps(manifest_entries[0], indent=2, default=str))

## 3. Ground Truth キーポイント形式の解析

Ground Truthの `crowd-keypoint` 出力形式:
```json
{
  "source-ref": "s3://bucket/key",
  "keypoints": {
    "annotations": [
      {
        "class_id": 0,
        "width": 10,
        "height": 10,
        "top": 100,
        "left": 200
      }
    ],
    "image_size": [{"width": 640, "height": 480, "depth": 3}]
  },
  "keypoints-metadata": {
    "objects": [{"confidence": 0.95}],
    "class-map": {"0": "needle_tip", "1": "needle_center", ...},
    "type": "groundtruth/keypoint"
  }
}
```

In [ ]:
def parse_ground_truth_keypoints(entry, label_attribute='keypoints'):
    """Ground Truthエントリからキーポイント情報を抽出
    
    Returns:
        dict: {
            'source_ref': str,
            'image_width': int,
            'image_height': int,
            'keypoints': {
                'needle_tip': (x, y, visibility),
                'needle_center': (x, y, visibility),
                'scale_min': (x, y, visibility),
                'scale_max': (x, y, visibility)
            }
        }
    """
    source_ref = entry['source-ref']
    annotation_data = entry.get(label_attribute, {})
    metadata = entry.get(f'{label_attribute}-metadata', {})
    
    # 画像サイズ
    image_size = annotation_data.get('image_size', [{}])[0]
    img_w = image_size.get('width', 0)
    img_h = image_size.get('height', 0)
    
    # クラスマップ (class_id -> label)
    class_map = metadata.get('class-map', {})
    
    # キーポイント抽出
    keypoints = {}
    annotations = annotation_data.get('annotations', [])
    
    for ann in annotations:
        class_id = str(ann.get('class_id', ''))
        label = class_map.get(class_id, f'unknown_{class_id}')
        
        # キーポイントの中心座標を計算（left, top はバウンディングボックスの左上）
        x = ann.get('left', 0) + ann.get('width', 0) / 2
        y = ann.get('top', 0) + ann.get('height', 0) / 2
        visibility = 2  # visible
        
        keypoints[label] = (x, y, visibility)
    
    return {
        'source_ref': source_ref,
        'image_width': img_w,
        'image_height': img_h,
        'keypoints': keypoints
    }

# テストパース
if manifest_entries:
    sample = parse_ground_truth_keypoints(manifest_entries[0])
    print(f'Source: {sample["source_ref"]}')
    print(f'Image size: {sample["image_width"]}x{sample["image_height"]}')
    for label, (x, y, v) in sample['keypoints'].items():
        print(f'  {label}: ({x:.1f}, {y:.1f}, vis={v})')

## 4. YOLOv8-pose 形式への変換

### YOLOv8-pose ラベルフォーマット
```
<class_id> <cx> <cy> <w> <h> <kp1_x> <kp1_y> <kp1_v> <kp2_x> <kp2_y> <kp2_v> ...
```
- 座標は画像サイズで正規化（0.0〜1.0）
- `class_id`: 0（メーター）
- `cx, cy, w, h`: バウンディングボックス（キーポイントを囲む矩形）
- `kp_x, kp_y`: キーポイント座標（正規化）
- `kp_v`: 可視性（0=不可視, 1=遮蔽, 2=可視）

In [ ]:
def compute_bounding_box(keypoints_dict, img_w, img_h, padding_ratio=0.1):
    """キーポイントを囲むバウンディングボックスを計算（正規化座標）
    
    Args:
        keypoints_dict: {label: (x, y, v)}
        img_w: 画像幅
        img_h: 画像高さ
        padding_ratio: パディング（短辺に対する比率）
    
    Returns:
        (cx, cy, w, h): 正規化されたバウンディングボックス
    """
    xs = [kp[0] for kp in keypoints_dict.values()]
    ys = [kp[1] for kp in keypoints_dict.values()]
    
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    
    # パディング追加
    pad_x = (x_max - x_min) * padding_ratio
    pad_y = (y_max - y_min) * padding_ratio
    
    x_min = max(0, x_min - pad_x)
    x_max = min(img_w, x_max + pad_x)
    y_min = max(0, y_min - pad_y)
    y_max = min(img_h, y_max + pad_y)
    
    # YOLO形式（中心 + サイズ、正規化）
    cx = ((x_min + x_max) / 2) / img_w
    cy = ((y_min + y_max) / 2) / img_h
    w = (x_max - x_min) / img_w
    h = (y_max - y_min) / img_h
    
    return cx, cy, w, h


def convert_to_yolo_pose(parsed_entry):
    """パース済みエントリをYOLOv8-pose形式のラベル文字列に変換
    
    Returns:
        str: YOLO形式のラベル行
             "<class_id> <cx> <cy> <w> <h> <kp1_x> <kp1_y> <kp1_v> ..."
    """
    img_w = parsed_entry['image_width']
    img_h = parsed_entry['image_height']
    kps = parsed_entry['keypoints']
    
    if not kps or img_w == 0 or img_h == 0:
        return None
    
    # バウンディングボックス
    cx, cy, w, h = compute_bounding_box(kps, img_w, img_h)
    
    # キーポイント（定義順に並べる）
    kp_values = []
    for label in KEYPOINT_LABELS:
        if label in kps:
            x, y, v = kps[label]
            kp_values.extend([x / img_w, y / img_h, v])
        else:
            # キーポイントが欠損の場合
            kp_values.extend([0.0, 0.0, 0])
    
    # class_id = 0（メーター）
    parts = [0, cx, cy, w, h] + kp_values
    return ' '.join(f'{v:.6f}' if isinstance(v, float) else str(v) for v in parts)


# テスト変換
if manifest_entries:
    sample_parsed = parse_ground_truth_keypoints(manifest_entries[0])
    yolo_line = convert_to_yolo_pose(sample_parsed)
    print(f'YOLO形式: {yolo_line}')

## 5. 品質チェック & 不良データ除外

In [ ]:
def is_valid_annotation(parsed_entry):
    """アノテーションの品質チェック
    
    除外条件:
    - キーポイントが全て同じ座標（判別不能マーク）
    - 必須キーポイントの欠損
    - 画像サイズが0
    """
    if parsed_entry['image_width'] == 0 or parsed_entry['image_height'] == 0:
        return False, 'image_size_zero'
    
    kps = parsed_entry['keypoints']
    
    # 必須キーポイントの存在チェック
    required = ['needle_tip', 'needle_center']
    for label in required:
        if label not in kps:
            return False, f'missing_{label}'
    
    # 全て同じ座標チェック（判別不能マーク）
    coords = [(kp[0], kp[1]) for kp in kps.values()]
    if len(set(coords)) == 1:
        return False, 'all_same_position'
    
    # needle_tip と needle_center が同じ位置のチェック
    if 'needle_tip' in kps and 'needle_center' in kps:
        tip = kps['needle_tip']
        center = kps['needle_center']
        dist = ((tip[0] - center[0])**2 + (tip[1] - center[1])**2)**0.5
        if dist < 5:  # 5ピクセル未満は近すぎる
            return False, 'needle_tip_center_too_close'
    
    return True, 'ok'


# 全データの品質チェック
valid_entries = []
invalid_entries = []
invalid_reasons = {}

for entry in manifest_entries:
    parsed = parse_ground_truth_keypoints(entry)
    is_valid, reason = is_valid_annotation(parsed)
    
    if is_valid:
        valid_entries.append((entry, parsed))
    else:
        invalid_entries.append((entry, reason))
        invalid_reasons[reason] = invalid_reasons.get(reason, 0) + 1

print(f'有効データ: {len(valid_entries)}')
print(f'無効データ: {len(invalid_entries)}')
if invalid_reasons:
    print('\n除外理由:')
    for reason, count in invalid_reasons.items():
        print(f'  {reason}: {count}')

## 6. データ分割 (train / val / test)

In [ ]:
# シャッフル
random.seed(42)
random.shuffle(valid_entries)

n = len(valid_entries)
n_train = int(n * TRAIN_RATIO)
n_val = int(n * VAL_RATIO)

train_entries = valid_entries[:n_train]
val_entries = valid_entries[n_train:n_train + n_val]
test_entries = valid_entries[n_train + n_val:]

print(f'Train: {len(train_entries)}')
print(f'Val:   {len(val_entries)}')
print(f'Test:  {len(test_entries)}')

## 7. YOLOv8-pose ディレクトリ構造の作成

```
dataset/
├── images/
│   ├── train/
│   ├── val/
│   └── test/
├── labels/
│   ├── train/
│   ├── val/
│   └── test/
└── dataset.yaml
```

In [ ]:
# ディレクトリ作成
for split in ['train', 'val', 'test']:
    (WORK_DIR / 'images' / split).mkdir(parents=True, exist_ok=True)
    (WORK_DIR / 'labels' / split).mkdir(parents=True, exist_ok=True)


def process_split(entries, split_name):
    """指定splitのデータを処理（画像DL + ラベル生成）"""
    processed = 0
    errors = 0
    
    for idx, (entry, parsed) in enumerate(entries):
        source_ref = parsed['source_ref']
        
        # S3 URIをパース
        # s3://bucket/key -> bucket, key
        s3_parts = source_ref.replace('s3://', '').split('/', 1)
        bucket = s3_parts[0]
        key = s3_parts[1]
        
        # ファイル名（S3キーからユニークな名前を生成）
        # images/company_id/device_id/YYYY/MM/DD/HHMMSS.jpg -> company_id_device_id_YYYYMMDD_HHMMSS.jpg
        key_parts = key.split('/')
        if len(key_parts) >= 7:
            file_name = f'{key_parts[1]}_{key_parts[2]}_{key_parts[3]}{key_parts[4]}{key_parts[5]}_{key_parts[6]}'
        else:
            file_name = key.replace('/', '_')
        
        base_name = Path(file_name).stem
        img_path = WORK_DIR / 'images' / split_name / file_name
        label_path = WORK_DIR / 'labels' / split_name / f'{base_name}.txt'
        
        try:
            # 画像ダウンロード
            s3.download_file(bucket, key, str(img_path))
            
            # ラベル生成
            yolo_line = convert_to_yolo_pose(parsed)
            if yolo_line:
                label_path.write_text(yolo_line + '\n')
                processed += 1
            else:
                # ラベル変換失敗 → 画像も削除
                img_path.unlink(missing_ok=True)
                errors += 1
        except Exception as e:
            errors += 1
            if idx < 3:  # 最初の3件だけエラー表示
                print(f'  Error [{idx}]: {e}')
    
    return processed, errors


print('データ処理開始...')
for split_name, entries in [('train', train_entries), ('val', val_entries), ('test', test_entries)]:
    print(f'\n{split_name}:')
    processed, errors = process_split(entries, split_name)
    print(f'  成功: {processed}, エラー: {errors}')

print('\n処理完了')

## 8. dataset.yaml 生成

In [ ]:
dataset_yaml = f"""# FactEye Meter Keypoint Dataset
# YOLOv8-pose format

path: /tmp/yolo_dataset
train: images/train
val: images/val
test: images/test

# クラス定義
names:
  0: meter

# キーポイント定義
kpt_shape: [{NUM_KEYPOINTS}, 3]  # [キーポイント数, (x, y, visibility)]
"""

yaml_path = WORK_DIR / 'dataset.yaml'
yaml_path.write_text(dataset_yaml)
print(f'dataset.yaml 生成完了: {yaml_path}')
print()
print(dataset_yaml)

## 9. S3にアップロード

In [ ]:
import subprocess

S3_DATASET_PREFIX = 'yolo-pose/dataset'
S3_DATASET_URI = f's3://{SAGEMAKER_BUCKET}/{S3_DATASET_PREFIX}'

# aws s3 sync でアップロード
cmd = f'aws s3 sync {WORK_DIR} {S3_DATASET_URI} --quiet'
print(f'アップロード中: {cmd}')
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print(f'アップロード完了: {S3_DATASET_URI}')
else:
    print(f'エラー: {result.stderr}')

## 10. データセット統計の確認

In [ ]:
# ローカルファイル数の確認
for split in ['train', 'val', 'test']:
    img_count = len(list((WORK_DIR / 'images' / split).glob('*.jpg')))
    label_count = len(list((WORK_DIR / 'labels' / split).glob('*.txt')))
    print(f'{split}: images={img_count}, labels={label_count}')
    
    if img_count != label_count:
        print(f'  ⚠ 画像とラベルの数が一致しません！')

In [ ]:
# ラベルファイルのサンプル確認
sample_labels = list((WORK_DIR / 'labels' / 'train').glob('*.txt'))[:3]
for label_file in sample_labels:
    print(f'\n{label_file.name}:')
    print(f'  {label_file.read_text().strip()}')

## 次のステップ

データセットの準備が完了しました。**03_training.ipynb** でYOLOv8n-poseモデルの学習を実行します。

### 使用する情報
- データセットS3パス: `s3://<SAGEMAKER_BUCKET>/yolo-pose/dataset/`
- ローカルパス: `/tmp/yolo_dataset/`
- dataset.yaml: `/tmp/yolo_dataset/dataset.yaml`